In [0]:
このデータは32×32×3のカラー画像で、10個のクラスから構成されています。
飛行機、自動車、鳥、猫、鹿、犬、カエル、ウマ、船、トラックの10個のクラスからなります。まずはそれを見てみます。

実はカラー画像の分類というのは、ディープラーニングを使わなくても従来の機械学習の手法でできます。
MNISTと同じように画像をベクトルにしてしまえばいいのです。減色操作をしてもいいですが、
しなくてもできます（しかも減色操作をしないほうがうまくいきます）。

例えばMNISTの場合、28×28×1（白黒なので1チャンネル）を784次元のベクトルとしました。
CIFAR-10の場合は32×32×3（RGBなので3チャンネル）を3072次元のベクトルとします。つまりベクトルを見ると、

[(0, 0)の赤], [(0, 1)の赤], ……, [(31, 31)の赤], [(0, 0)の緑], ……, [(31, 31)の緑], ……, [(31, 31)の青] 

In [1]:
# -*- coding: utf-8 -*-
#-*- coding: cp950 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from typing import *

import torch
import torch.optim as optim

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *
from fastai.core import *

import tensorflow as tf
import random as rn
np.random.seed(0)
rn.seed(0)

import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle

from keras.preprocessing import image
from sklearn.model_selection import train_test_split

# Importing Keras libraries
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
 
import warnings
warnings.filterwarnings('ignore')
import cv2

from keras.applications import ResNet50
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
os.listdir()

['.config', 'sample_data']

In [3]:
#在colab下接入google drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive'
#path = Path(base_dir +'/imgs4')  #imgs
path = Path(base_dir +'/DLtry')
path.mkdir(parents=True,exist_ok=True)
os.chdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# データの読み込み
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 小数化
x_train = x_train / 255
x_test = x_test / 255

170500096/170498071 [==============================] - 2s 0us/step


In [5]:
# ロジスティック回帰
# データ数
m_train, m_test = x_train.shape[0], x_test.shape[0]

# ベクトル化
x_train, x_test = x_train.reshape(m_train, -1), x_test.reshape(m_test, -1)

# ノルムで標準化*******
x_train2 = x_train / np.linalg.norm(x_train, ord=2, axis=1, keepdims=True)
x_test2 = x_test / np.linalg.norm(x_test, ord=2, axis=1, keepdims=True)

import time
start_time = time.time()
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression()

logr.fit(x_train2, y_train)
print("Elapsed[s] : ", time.time() - start_time)
print("Train :", logr.score(x_train2, y_train))
print("Test :", logr.score(x_test2, y_test))

Elapsed[s] :  93.41054677963257
Train : 0.4171
Test : 0.4057


In [0]:
model=logr.fit(x_train2, y_train)
print(model.intercept_ )    #下列第一橫列
print(model.coef_) 

[-5.536812  6.177111 -2.328945  4.565471 -5.383747  1.766808 -1.933553  3.903827 -6.376968  5.146806]
[[ 0.170352  0.110942  0.982344  0.061879 ... -0.437402 -0.239735 -0.529524 -0.458354]
 [-0.135657 -0.470005 -0.585361  0.067784 ...  0.306831 -0.073056 -0.107496  0.312171]
 [ 0.089479  0.246642 -0.789732  0.282404 ... -0.344885  0.102758 -0.025665 -0.451987]
 [-0.055889 -0.653467 -0.851693  0.080396 ...  0.005711 -0.734209 -0.992213 -0.124824]
 ...
 [ 0.425854  0.274827 -0.631068  0.351925 ...  0.32477   0.430599  0.276689  0.385682]
 [-0.32784   0.15587   0.831972 -0.49632  ... -0.97884   1.002425  0.842056 -0.739654]
 [ 0.72591   0.620631  1.093757  0.564045 ...  0.943361 -1.527826 -0.264917  1.011463]
 [-0.127313  0.498678  1.301489 -0.324359 ...  0.777317  1.274991  0.645573  0.755904]]


In [0]:
pd.DataFrame(model.coef_)   #10個橫列,代表10種類別,用SVD或PCA來降維度

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.170352,0.110942,0.982344,0.061879,-0.029149,0.821731,-0.067688,-0.156417,0.679870,-0.023088,-0.095084,0.694987,-0.000712,-0.054653,0.733647,0.038419,-0.026671,0.773216,-0.013351,-0.048784,0.743241,-0.068644,-0.104675,0.668023,-0.086362,-0.148650,0.593770,-0.150440,-0.194674,0.539340,-0.160521,-0.222008,0.497614,-0.081703,-0.158802,0.544295,-0.086241,-0.169274,0.535335,-0.113950,...,0.190958,-0.226537,-0.175595,0.042184,-0.155724,-0.155747,0.065615,-0.148210,-0.161198,0.030690,-0.202694,-0.239865,-0.047045,-0.169441,-0.233827,-0.039532,-0.198519,-0.303066,-0.121805,-0.201119,-0.345192,-0.183783,-0.206941,-0.363356,-0.230506,-0.161588,-0.343015,-0.234186,-0.127824,-0.340769,-0.233779,-0.233662,-0.462781,-0.357761,-0.263384,-0.517860,-0.437402,-0.239735,-0.529524,-0.458354
1,-0.135657,-0.470005,-0.585361,0.067784,-0.274115,-0.420352,0.177769,-0.179553,-0.337759,0.128369,-0.264328,-0.414132,0.210179,-0.173780,-0.369065,0.063661,-0.273589,-0.489345,0.119159,-0.267662,-0.525221,0.148159,-0.282672,-0.543138,0.168090,-0.262786,-0.516317,0.238079,-0.199010,-0.508002,0.235430,-0.144188,-0.463871,0.291300,-0.068481,-0.380031,0.487716,0.148166,-0.153840,0.348827,...,0.342376,-0.134772,0.112849,0.414571,-0.163739,0.106292,0.385559,-0.200119,0.052170,0.365080,-0.208472,0.007526,0.361016,-0.256804,-0.040545,0.283839,-0.205116,-0.017613,0.288209,-0.113846,0.045281,0.380295,-0.100708,0.025574,0.373895,-0.096852,0.003800,0.338331,-0.137719,-0.060964,0.289291,-0.127755,-0.089638,0.308121,-0.109431,-0.113212,0.306831,-0.073056,-0.107496,0.312171
2,0.089479,0.246642,-0.789732,0.282404,0.482424,-0.569888,0.362358,0.612738,-0.407440,0.268834,0.494552,-0.470589,0.203351,0.423266,-0.554222,0.283557,0.500828,-0.495258,0.245827,0.458104,-0.482191,0.139352,0.354100,-0.575857,-0.027552,0.207872,-0.751808,-0.076855,0.190976,-0.758933,-0.067200,0.214198,-0.712850,-0.058585,0.178959,-0.719622,-0.110560,0.194202,-0.715525,-0.007394,...,-0.286348,0.237421,-0.448279,-0.269281,0.227421,-0.381945,-0.280927,0.323982,-0.216069,-0.121551,0.461559,-0.062167,-0.044077,0.466828,-0.001314,-0.021455,0.365594,-0.025422,-0.082545,0.069434,-0.192250,-0.319904,0.057766,-0.165663,-0.324839,0.172812,-0.022306,-0.205564,0.192265,0.006255,-0.212634,0.263725,0.072490,-0.237993,0.240946,0.065739,-0.344885,0.102758,-0.025665,-0.451987
3,-0.055889,-0.653467,-0.851693,0.080396,-0.508936,-0.720090,0.033661,-0.571098,-0.752591,-0.045136,-0.605190,-0.756925,-0.088443,-0.646124,-0.756939,-0.016439,-0.611492,-0.726793,0.094802,-0.538317,-0.675778,0.245769,-0.376066,-0.531553,0.165936,-0.422483,-0.571585,0.201140,-0.386666,-0.481207,0.255422,-0.368411,-0.489501,0.324441,-0.269212,-0.409998,0.283755,-0.293444,-0.452560,0.363977,...,0.360703,0.101410,-0.283962,0.284754,0.111853,-0.300922,0.306454,0.078323,-0.364084,0.257522,0.110083,-0.310958,0.312865,0.127691,-0.298495,0.372475,-0.046637,-0.439122,0.269240,-0.193006,-0.549424,0.190758,-0.153938,-0.485331,0.246882,0.023121,-0.276256,0.489237,-0.083270,-0.393634,0.412717,-0.398042,-0.682134,0.151779,-0.552188,-0.826852,0.005711,-0.734209,-0.992213,-0.124824
4,-0.581399,-0.557312,-0.694070,-0.290720,-0.218851,-0.398694,-0.098784,-0.063247,-0.243180,-0.038698,0.006200,-0.147390,0.072529,0.184546,0.048467,0.008363,0.182948,0.052323,0.018251,0.242863,0.071594,0.015752,0.262113,0.054608,0.109772,0.347657,0.155501,0.187320,0.414248,0.194667,0.149837,0.365032,0.148198,0.151677,0.373214,0.143242,0.052412,0.273287,0.005243,-0.039143,...,-0.735168,0.199430,0.774656,-0.651895,0.286053,0.864530,-0.542111,0.347001,0.904555,-0.509334,0.331985,0.854376,-0.558608,0.397627,0.853825,-0.571223,0.336927,0.801008,-0.590190,0.411376,0.892589,-0.559042

In [6]:
#SVM 未標準化(有時優於有標準化)
from sklearn.svm import LinearSVC
svc = LinearSVC()

svc.fit(x_train2, y_train)
print("Elapsed[s] : ", time.time() - start_time)
print("Train :", svc.score(x_train2, y_train))
print("Test :", svc.score(x_test2, y_test))

Elapsed[s] :  845.5834405422211
Train : 0.45122
Test : 0.4062


In [0]:
#GB
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier()

GB.fit(x_train2, y_train)
print("Elapsed[s] : ", time.time() - start_time)
print("Train :", GB.score(x_train2, y_train))
print("Test :", GB.score(x_test2, y_test))

In [0]:
#RF
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(x_train, y_train)
print("Elapsed[s] : ", time.time() - start_time)
print("Train :", rf.score(x_train, y_train))
print("Test :", rf.score(x_test, y_test))

Elapsed[s] :  421.99188804626465
Train : 1.0
Test : 0.4646


In [0]:
#設定max_depth,未必較佳
rf = RandomForestClassifier(max_depth=8)

rf.fit(x_train2, y_train)
print("Elapsed[s] : ", time.time() - start_time)
print("Train :", rf.score(x_train2, y_train))
print("Test :", rf.score(x_test2, y_test))

Elapsed[s] :  855.7198085784912
Train : 0.48894
Test : 0.4065


In [0]:
clf=rf.fit(x_train, y_train)
clf.feature_importances_

array([0.000295, 0.000296, 0.000444, 0.000286, ..., 0.000355, 0.000461, 0.000297, 0.00037 ])

In [0]:
pd.DataFrame(list(zip(x_train, clf.feature_importances_)))   #PCA,NMF

,0,1
0,"[0.23137254901960785, 0.24313725490196078, 0.2...",0.000295
1,"[0.6039215686274509, 0.6941176470588235, 0.733...",0.000296
2,"[1.0, 1.0, 1.0, 0.9921568627450981, 0.99215686...",0.000444
3,"[0.10980392156862745, 0.09803921568627451, 0.0...",0.000286
4,"[0.6666666666666666, 0.7058823529411765, 0.776...",0.000284
...,...,...
3067,"[0.5137254901960784, 0.16862745098039217, 0.29...",0.000342
3068,"[0.6627450980392157, 0.7019607843137254, 0.749...",0.000355
3069,"[0.5098039215686274, 0.11372549019607843, 0.05...",0.000461
3070,"[0.21568627450980393, 0.2549019607843137, 0.22...",0.000297
